In [ ]:
import torch
from torch import nn
import numpy as np
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
import os

In [2]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
device

device(type='cuda', index=0)

In [ ]:
train_dir = 'dogs-vs-cats/train/'
dog_dir = train_dir + 'dog/'
cat_dir = train_dir + 'cat/'
os.makedirs(dog_dir, exist_ok=True)
os.makedirs(cat_dir, exist_ok=True)

In [ ]:
for filename in os.listdir(train_dir):
    if 'dog.' in filename.lower():
        os.rename(os.path.join(train_dir, filename), os.path.join(dog_dir, filename))
    if 'cat.' in filename.lower():
        os.rename(os.path.join(train_dir, filename), os.path.join(cat_dir, filename))

In [5]:
data_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [6]:
train_ds = ImageFolder(root=train_dir, transform=data_transforms)

In [7]:
train_ds, valid_ds = torch.utils.data.random_split(train_ds, [0.8, 0.2])

In [8]:
mini_batch_size = 512
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=mini_batch_size, shuffle=True, drop_last=False)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=mini_batch_size)

In [9]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        for b in self.dl:
            yield (self.func(*b))


def put_to_gpu(x, y):
    return x.to(device), y.to(device)

In [10]:
model = models.alexnet(pretrained=True)
# model = models.vgg16(pretrained=True)

c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\70p6229\computer_vision\examples\pytorch\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
for param in model.parameters():
    param.requires_grad = False

In [12]:
model.classifier[6].out_features = 2

In [13]:
for param in model.classifier.parameters():
    param.requires_grad = True

In [14]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

58631144

In [15]:
model = model.to(device)

In [16]:
optimizer = torch.optim.Adam(model.parameters())

In [17]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_state = None

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict()
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict()
            self.counter = 0

    def load_best_model(self, model):
        model.load_state_dict(self.best_model_state)

In [18]:
early_stopping = EarlyStopping(patience=3, delta=0.01)

In [19]:
def fit(epochs, model, optimizer, train_dl, valid_dl=None):
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()

        for X_mb, y_mb in train_dl:
            y_hat = model(X_mb)

            loss = loss_func(y_hat, y_mb)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        with torch.no_grad():
            train_loss = sum(loss_func(model(X_mb), y_mb) for X_mb, y_mb in train_dl)
            valid_loss = sum(loss_func(model(X_mb), y_mb) for X_mb, y_mb in valid_dl)
        print('epoch {}, training loss {}'.format(epoch + 1, train_loss / len(train_dl)))
        print('epoch {}, validation loss {}'.format(epoch + 1, valid_loss / len(valid_dl)))

        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    print('Finished training')

    return model

In [20]:
epochs = 10

model = fit(epochs, model, optimizer, WrappedDataLoader(train_dl, put_to_gpu), WrappedDataLoader(valid_dl, put_to_gpu))

epoch 1, training loss 0.22537605464458466
epoch 1, validation loss 0.21992981433868408
epoch 2, training loss 0.1812870353460312
epoch 2, validation loss 0.19846127927303314
epoch 3, training loss 0.1920304149389267
epoch 3, validation loss 0.2211572378873825
epoch 4, training loss 0.16464878618717194
epoch 4, validation loss 0.1918969452381134
epoch 5, training loss 0.15724538266658783
epoch 5, validation loss 0.19126954674720764
epoch 6, training loss 0.1360851526260376
epoch 6, validation loss 0.18414847552776337
epoch 7, training loss 0.11697882413864136
epoch 7, validation loss 0.17918610572814941
epoch 8, training loss 0.09142089635133743
epoch 8, validation loss 0.15652766823768616
epoch 9, training loss 0.07952514290809631
epoch 9, validation loss 0.15518808364868164
epoch 10, training loss 0.07688505202531815
epoch 10, validation loss 0.16821031272411346
Finished training


In [21]:
early_stopping.load_best_model(model)
model = model.cpu()

In [22]:
def evaluate(model, data_loader):    
    model.eval()
    accuracy = 0
    with torch.no_grad():
        for X, y in data_loader:
            y_hat = model(X).cpu().numpy()
            y_hat = np.argmax(y_hat, axis=1)
            accuracy += (y_hat == y.cpu().numpy()).mean()
    accuracy /= len(data_loader)

    return accuracy

In [23]:
evaluate(model.to(device), WrappedDataLoader(train_dl, put_to_gpu))

0.9716796875

In [24]:
evaluate(model.to(device), WrappedDataLoader(valid_dl, put_to_gpu))

0.9347576530612244